In [ ]:
import json
import csv
import requests
import boto3
from my_aws_secrets import s3_access_key_id, s3_secret_access_key

headers = {
    'authority': 'api.acrossthetimeline.com',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'content-type': 'text/plain;charset=UTF-8',
    'origin': 'https://acrossthetimeline.com',
    'referer': 'https://acrossthetimeline.com/',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36',
}

data = '{"queries":["SELECT CASE WHEN t.Name = \'San Antonio Stars\' OR t.Name = \'San Antonio Silver Stars\' THEN \'San Antonio Stars\' ELSE t.Name END AS Team, t.`Primary Color` AS Color, g.`Game Type`, g.Attendance, YEAR(g.`Start Timestamp`) AS Season, g.`Start Timestamp` AS Timestamp, o.Name AS Opponent, g.Arena, g.City, g.State, t.Abbreviation AS TeamAbb, o.Abbreviation AS OpponentAbb FROM Games g JOIN Teams t ON t.ID = g.`Home Team ID` JOIN Teams o ON o.ID = g.`Away Team ID` WHERE YEAR(g.`Start Timestamp`) != 2020 AND Attendance IS NOT NULL AND `Game Type` != \'Preseason\' ORDER BY g.`Start Timestamp`"]}'

# response = requests.post('https://api.acrossthetimeline.com/data', headers=headers, data=data)


def main():
    get_wnba_games_attendance()
    load_data_to_s3_bucket()
    
    
wnba_attendance_list = []


def get_wnba_games_attendance():
    response = requests.post('https://api.acrossthetimeline.com/data', headers=headers, data=data)
    # pprint(response.json())
    # clean the data - iterate over the dataset to pull only the necessary fields
    for dataset in response.json()["values"][0]:
        dataset_dict = dict((dataset_keys.lower(), dataset_values) for dataset_keys, dataset_values in dataset.items())
        # remove unnecessary columns
        dataset_dict.pop("color")
        dataset_dict.pop("timestamp")
        # rename column names for consistency
        dataset_dict["game_type"] = dataset_dict.pop("game type")
        dataset_dict["team_abb"] = dataset_dict.pop("teamabb")
        dataset_dict["opponent_abb"] = dataset_dict.pop("opponentabb")
        
        # print(dataset_dict)
        
        # append the dataset dicts to the list in order to load to json and csv file
        wnba_attendance_list.append(dataset_dict)

    print(wnba_attendance_list)
    # transform the data into json
    wnba_attendance_json = json.dumps(wnba_attendance_list, indent=4)
    print(wnba_attendance_json)
    
    # create a json file and load the attendance data
    with open("wnba_attendance.json", "w") as wnba_attendance_file:
        wnba_attendance_file.write(wnba_attendance_json)
        wnba_attendance_file.close()
    
    # create a csv file and load the attendance data
    wnba_attendance_csv = open("wnba_attendance.csv", "w", newline="")
    csv_writer = csv.writer(wnba_attendance_csv)
    count = 0

    for game in wnba_attendance_list:
        if count == 0:
            header = game.keys()
            csv_writer.writerow(header)
            count += 1
        csv_writer.writerow(game.values())

    wnba_attendance_csv.close()


def load_data_to_s3_bucket():
    
    print(f" access key id --> {s3_access_key_id}")
    print(f"access key --> {s3_secret_access_key}")
    # create a client to access resources with
    s3_client = boto3.Session(
            #     's3', # add the service name we are accessing
            aws_access_key_id=s3_access_key_id,  # update this
            aws_secret_access_key=s3_secret_access_key  # update this
            )
    s3 = s3_client.resource('s3')
    # print(f"s3 bucket info {s3_client.list_buckets()['Buckets']}")
    # print(s3_client.list_objects(Bucket='ds4a-ec1-s3-bucket'))
    # upload the applications to the bucket created earlier
    # we have a try/except clause to see if it fails
    try:
        # s3_client.upload_file('data/empowerment_apps_updated.csv', "ds4a-ec1-s3-bucket/ds4a-recruiting/",s3_access_key_id)
        s3.Bucket('data-load-bucket-attendance').upload_file('wnba_attendance.csv',
                                                    'wnba_attendance.csv')
        print('File successfully loaded')
    except:
        print('Client error: file could not be uploaded to S3')
        

if __name__ == '__main__':
    main()